In [13]:
import warnings
warnings.filterwarnings("ignore")


import numpy as np
from numba import jit,cuda
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32
from matplotlib import pyplot as plt
import math,time
import scipy.ndimage.filters
from threading import Thread,Lock
from ipywidgets import Image, HBox
import ipywidgets as widgets
from ipycanvas import Canvas
import io
x = np.ones([5,5])/np.zeros([5,5])
#print(x)



class Cleanup():
    def __init__(self):
        self.threads = []
    def add(self,*threads):
        for thread in threads:
            self.threads.append(thread)
    def reset(self):
        for t in self.threads:
            t.isalive=False
            t.join()
        self.threads=[]
    def hard_reset(self):
        for a in self.threads:
            try:
                a.start()
            except:
                pass
        self.reset()
cleaner = Cleanup()

@jit(nopython=True)
def update(V,p,mutable,w,h=1):
    #assume p is predivided by epsilon
    #V2=V.copy()
    sx,sy=V.shape
    rmax=0
    for i in range(sx):
        for j in range(sy):
            R = (h**2*p[i][j]+V[i-1][j]+V[i+1-sx][j]+V[i][j-1]+V[i][j+1-sy])/4-V[i][j]
            if mutable[i][j]:
                V[i][j]+=w*R
                if abs(R)>rmax:
                    rmax=abs(R)
    return V,rmax

@cuda.jit
def fast_ising(grid,JB,mew,parity,rng_states,size):
    """Updates 1/4 of the grid by stochastically choosing points
    on a checkerboard grid and updating with boltzmann probability"""
    #find grid location
    i,j=cuda.grid(2)
    #get your first random number
    n = xoroshiro128p_uniform_float32(rng_states, i*size+j)
    #parity is 0 or 1, and tells you which half of the checkerboard to update
    if ((i+j)%2==parity and n < 0.5 and i<size and j<size):
        #sum the values of the nearest neighbors
        sum_=grid[i-1][j]+grid[i+1-size][j]+grid[i][j-1]+grid[i][j+1-size]
        #its grid*2-1 because I am using zero for spin down and 1 for spin up
        delta = 2.0 * (grid[i][j]*2-1)*(2*sum_-4+mew)
        #get our second random number
        n = xoroshiro128p_uniform_float32(rng_states, i*size+j)
        #update the cell according to the boltzmann distribution
        if (delta < 0 or 1-n < math.exp(-JB*delta)):
            grid[i][j]=1-grid[i][j]
            
@cuda.jit
def updategpu(V,p,mutable,w,h,parity):
    #assume p is predivided by epsilon
    i, j = cuda.grid(2)
    sx,sy=V.shape    
    if mutable[i][j] and ((i+j)%2==parity) and i<sx and j<sy:
        R = (h**2*p[i][j]+V[i-1][j]+V[i+1-sx][j]+V[i][j-1]+V[i][j+1-sy])/4.0-V[i][j]
        V[i][j]+=w*R
        
@cuda.jit
def generalpoisson(V,p,eps,mutable,w,h,parity):
    #assume p is predivided by epsilon_0
    #eps gives the multiplier by epsilon_0 of the permitivity and is staggered wrto V and p
    i, j = cuda.grid(2)
    sx,sy=V.shape
    if mutable[i][j] and ((i+j)%2==parity) and i<sx and j<sy:
        a0=eps[i][j]+eps[i-1][j]+eps[i][j-1]+eps[i-1][j-1]
        a1=(eps[i][j]+eps[i][j-1])/2.0
        a2=(eps[i-1][j]+eps[i][j])/2.0
        a3=(eps[i-1][j-1]+eps[i-1][j])/2.0
        a4=(eps[i][j-1]+eps[i-1][j-1])/2.0
        R = (h**2*p[i][j]+a1*V[i+1-sx][j]+a2*V[i][j+1-sy]+a3*V[i-1][j]+a4*V[i][j-1])/a0-V[i][j]
        V[i][j]+=w*R        

@cuda.jit
def generalpoisson_err(V,p,eps,mutable,w,h,arr_answer):
    #assume p is predivided by epsilon_0
    #eps gives the multiplier by epsilon_0 of the permitivity and is staggered wrto V and p
    i, j = cuda.grid(2)
    sx,sy=V.shape    
    if mutable[i][j] and i<sx and j<sy:
        a0=eps[i][j]+eps[i-1][j]+eps[i][j-1]+eps[i-1][j-1]
        a1=(eps[i][j]+eps[i][j-1])/2.0
        a2=(eps[i-1][j]+eps[i][j])/2.0
        a3=(eps[i-1][j-1]+eps[i-1][j])/2.0
        a4=(eps[i][j-1]+eps[i-1][j-1])/2.0
        R = (h**2*p[i][j]+a1*V[i+1-sx][j]+a2*V[i][j+1-sy]+a3*V[i-1][j]+a4*V[i][j-1])/a0-V[i][j]
        aR=abs(R)
        if aR>arr_answer[0]:
            arr_answer[0]=aR
        if aR>arr_answer[0]:
            arr_answer[0]=aR
        
@cuda.jit
def thread_unsafe_err(V,p,mutable,w,h,arr_answer):
    #assume p is predivided by epsilon
    i, j = cuda.grid(2)
    sx,sy=V.shape    
    if mutable[i][j] and i<sx and j<sy:
        R = (h**2*p[i][j]+V[i-1][j]+V[i+1-sx][j]+V[i][j-1]+V[i][j+1-sy])/4.0-V[i][j]
        aR=abs(R)
        if aR>arr_answer[0]:
            arr_answer[0]=aR
        if aR>arr_answer[0]:
            arr_answer[0]=aR
@cuda.jit
def setzero(ans):
    i=cuda.grid(1)
    if i<ans.size:
        ans[i]=0
        
        


In [2]:

class Ising(Thread):
    def __init__(self, N):
        self.N=N
        self.grid=np.asarray(np.random.random([N,N])>0.5,dtype=np.float32)
        self.threadsperblock = (16, 16)#should end up a multiple of 32 I think
        blockspergrid_x = int(np.ceil(self.grid.shape[0] / self.threadsperblock[0]))
        blockspergrid_y = int(np.ceil(self.grid.shape[1] / self.threadsperblock[1]))
        self.blockspergrid = (blockspergrid_x, blockspergrid_y)
        self.isalive=True
        self.rng_states = create_xoroshiro128p_states(self.grid.size, seed=1)
        self.grid_global_mem = cuda.to_device(self.grid)
        self.JB=5
        self.mew=0
        self.rspeed=100
        super(Ising, self).__init__()
    def run(self):
        while self.isalive:
            #parity=np.random.random()>0.5
            for x in range(self.rspeed):
                fast_ising[self.blockspergrid, self.threadsperblock](self.grid_global_mem,self.JB,self.mew,0,self.rng_states,self.N)
                fast_ising[self.blockspergrid, self.threadsperblock](self.grid_global_mem,self.JB,self.mew,1,self.rng_states,self.N)
            if self.rspeed<100:
                time.sleep(0.05)

class Render(Thread):
    def __init__(self, globalmem, canvas):
        self.grid_global_mem = globalmem
        self.canvas = canvas
        self.isalive=True
        super(Render, self).__init__()
    def run(self):
        while self.isalive:
            gridf= self.grid_global_mem.copy_to_host()
            
            blue_channel = (1-gridf)*255
            red_channel = gridf*255
            green_channel = np.zeros_like(gridf)
            image_data = np.stack((red_channel, green_channel, blue_channel), axis=2)
            self.canvas.put_image_data(image_data, 0, 0)
            time.sleep(0.01)

In [9]:
class V_GPU(Thread):
    def __init__(self, globalmem, canvas,boundary="zero",graph="E"):
        self.tol=1e-3
        self.grid_global_mem = globalmem
        self.graph=graph
        self.canvas = canvas
        self.isalive=True
        grid = (self.grid_global_mem.copy_to_host())*2-1
        V=np.random.random(grid.shape)*0
        self.threadsperblock = (16, 16)#should end up a multiple of 32 I think
        blockspergrid_x = int(np.ceil(V.shape[0] / self.threadsperblock[0]))
        blockspergrid_y = int(np.ceil(V.shape[1] / self.threadsperblock[1]))
        self.blockspergrid = (blockspergrid_x, blockspergrid_y)
        sx,sy=V.shape
        p=grid
        xp=int(0.36*sx)
        if boundary=='plate':
            V[xp,xp:-xp]=1
            V[-xp,xp:-xp]=-1
            mutable = V>-10
            mutable[xp,xp:-xp]=False
            mutable[-xp,xp:-xp]=False
        elif boundary=='zero':
            V[0][0]=0
            mutable = V>-10
            mutable[0][0]=False
        elif boundary=='mzero':
            V[sx//2,sy//2]=0
            mutable = V>-10
            mutable[sx//2,sy//2]=False
        elif boundary == 'zeros':
            V[0,:]=0
            V[:,0]=0
            mutable = V>-10
            mutable[0,:]=False
            mutable[:,0]=False
        elif boundary == 'top':
            V[0,:]=0
            mutable = V>-10
            mutable[0,:]=False
        elif boundary == 'yplate':
            V[0,:]=1
            V[-1,:]=-1
            mutable = V>-10
            mutable[0,:]=False
            mutable[-1,:]=False
        elif boundary == 'none':
            mutable = V>-10
        elif boundary == 'closed':
            V[0,:]=0
            V[:,0]=0
            V[-1,:]=0
            V[:,-1]=0
            mutable = V>-10
            mutable[0,:]=False
            mutable[:,0]=False
            mutable[-1,:]=False
            mutable[:,-1]=False
        t = np.cos(np.pi/sx)+np.cos(np.pi/sy)
        w = (8-np.sqrt(64-16*t**2))/t**2
        
        self.V=cuda.to_device(V)
        self.mutable=cuda.to_device(mutable)
        self.w=w
        super(V_GPU, self).__init__()
    def run(self):
        ans=cuda.to_device([0.0])
        while self.isalive:
            err=100
            p = (self.grid_global_mem.copy_to_host())*3-2
            p = cuda.to_device(p)
            i=0
            while err>1e-2:
                for x in range(100):
                    updategpu[self.blockspergrid, self.threadsperblock](self.V,p,self.mutable,self.w,0.01,0)
                    updategpu[self.blockspergrid, self.threadsperblock](self.V,p,self.mutable,self.w,0.01,1)
                thread_unsafe_err[self.blockspergrid, self.threadsperblock](self.V,p,self.mutable,self.w,0.01,ans)
                err = ans.copy_to_host()[0]
                setzero[1,1](ans)
            Vf=self.V.copy_to_host()
            if self.graph=='E':
                self.render_E(Vf)
            else:
                max_=np.max(Vf)
                min_=np.min(Vf)
                RGB = plt.cm.jet((Vf-min_)/(max_-min_))
                self.canvas.put_image_data(RGB[:,:,:3]*255, 0, 0)
            self.Vf=Vf
            #time.sleep(0.01)
    def render_E(self,V):
        s=30
        plt.rcParams['figure.figsize'] = [6.8,6.8]
        #pad the boundaries with pbc for better gradient at the boundaries
        V=np.pad(V,9,'wrap')
        v,u = np.gradient(-V)
        #remove the padded bits
        v=v[10:-10,10:-10]
        u=u[10:-10,10:-10]
        X=np.arange(v.shape[1])[::s]

        Y=np.arange(v.shape[0])[::s]
        X,Y = np.meshgrid(X,Y)
        m=(u**2+v**2)**0.5
        avg=np.sum(m)/m.size
        varm=np.sum(m**2)/m.size-avg**2
        stdv=varm**0.5
        self.stdv=stdv
        self.avg=avg
        scaler=np.max(m)**0.5+1e-8
        m = np.clip(m,avg-2*stdv,avg+2*stdv)
        #print(np.max(m),np.min(m),avg,varm)
        time.sleep(0.01)
        fig = plt.figure()
        axs = fig.add_subplot(111)
        axs.axis('off')
        axs.quiver(X,Y,u[::s,::s],-v[::s,::s],scale=scaler*1.5)
        axs.imshow(m,cmap='jet')
        with io.BytesIO() as buff:
            fig.savefig(buff, format='raw')
            fig.clf()
            plt.close()
            data = np.frombuffer(buff.getvalue(), dtype=np.uint8)
        w, h = fig.bbox.bounds[2:]
        data = data.reshape((int(h), int(w), -1))
        self.canvas.put_image_data(data[80:,85:], 0, 0)

class V_GPU2(V_GPU):
    def run(self):
        ans=cuda.to_device([0.0])
        while self.isalive:
            err=100
            grid = 2-(self.grid_global_mem.copy_to_host())
            p = cuda.to_device(np.zeros_like(self.V))
            eps=cuda.to_device(grid)
            i=0
            while err>self.tol:
                for x in range(100):
                    generalpoisson[self.blockspergrid, self.threadsperblock](self.V,p,eps,self.mutable,self.w,0.01,0)
                    generalpoisson[self.blockspergrid, self.threadsperblock](self.V,p,eps,self.mutable,self.w,0.01,1)
                generalpoisson_err[self.blockspergrid, self.threadsperblock](self.V,p,eps,self.mutable,self.w,0.01,ans)
                err = ans.copy_to_host()[0]
                setzero[1,1](ans)
            Vf=self.V.copy_to_host()
            if self.graph=='E':
                self.render_E(Vf)
            else:
                max_=np.max(Vf)
                min_=np.min(Vf)
                RGB = plt.cm.jet((Vf-min_)/(max_-min_))
                self.canvas.put_image_data(RGB[:,:,:3]*255, 0, 0)
            self.Vf=Vf

In [10]:
%matplotlib agg
%matplotlib agg
image_data=np.zeros([512,512,4])
canvas = Canvas(width=image_data.shape[0], height=image_data.shape[1])
canvas2 = Canvas(width=image_data.shape[0], height=image_data.shape[1])
canvas2.put_image_data(image_data, 0, 0)         
            
model = Ising(512)
rend = Render(model.grid_global_mem,canvas)
voltage=V_GPU2(model.grid_global_mem,canvas2,'yplate','E')

cleaner.add(model,rend,voltage)
model.start()
rend.start() 
voltage.start()

i=0
def func(Beta,mew):
    model.JB=Beta
    model.mew=mew
plswork = widgets.Layout(width='50%')
x = widgets.FloatSlider(min=0,max=5,value=1,step=0.001,layout=plswork)
x.style.handle_color = 'lightblue'
y = widgets.FloatSlider(min=-2,max=2,step=0.001,layout=plswork)
widgets.interact(func,Beta=x,mew=y)
HBox([canvas,canvas2])  

interactive(children=(FloatSlider(value=1.0, description='Beta', layout=Layout(width='50%'), max=5.0, step=0.0…

In [11]:
model.rspeed=50

In [12]:
cleaner.reset()